### 1. Load data

In [ ]:
library(tidyverse)
options(repr.plot.width=8, repr.plot.height=6)

summary <- read_csv("summary.csv") # load summary data
meta <- read_csv("metadata.txt") # load metadata
meta[1:5,] # print the first 5 lines to know how to parse metadata

### 2. Convert metadata

In [ ]:
# add required columns (filename, volume and comments) from metadata
file <- paste0(meta$sample,".fcs") # format  sample name to filename (.fcs)
volume <- meta$volume
comments <- meta$comments

# add required metadata for CMAP
time <- as.POSIXct(meta$date, format="%d/%b/%y", tz="UTC") 
lat <- NA
lon <- NA

# add key information from sample label
label <- matrix(unlist(list(strsplit(meta$label, split=" "))), ncol=3, byrow=T) 
treatment <- label[,1]
timepoint <- label[,2]
replicate <- label[,3]

# create new metadata
metadata <- tibble(file, time, lat, lon, volume, treatment, timepoint, replicate, comments)

### 3. Add metadata to summary data

In [ ]:
all <- merge(summary, metadata, by="file")

all$abundance <- all$count * all$volume # calculate cell abundance (cells/microliter)

### 4. Plotting
#### a. Abundance

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(timepoint, population, treatment) %>%
    summarize(sd = sd(abundance),
              avg=mean(abundance)) %>%
    ggplot(aes(timepoint, avg, group=treatment, col=treatment)) +
    geom_point(size=3) +
    geom_line() +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Abundance (cells uL-1)")

#### b. Scatter

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(timepoint, population, treatment) %>%
    summarize(sd = sd(scatter),
              avg=mean(scatter)) %>%
    ggplot(aes(timepoint, avg, group=treatment, col=treatment)) +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    geom_point(size=3) +
    geom_line() +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Scatter (normalized to beads)")

#### c. Red fluorescence

In [ ]:
all %>%
    filter(population != "unknown" & population != "beads") %>%
    group_by(timepoint, population, treatment) %>%
    summarize(sd = sd(red),
              avg=mean(red)) %>%
    ggplot(aes(timepoint, avg, group=treatment, col=treatment)) +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    geom_point(size=3) +
    geom_line() +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Red fluo (normalized to beads)")

### 5. Save data

In [ ]:
library(FCSplankton)
library(openxlsx)
#source("meta.R") # template

project <- basename(getwd())
cruise <- "KM1906" # set to NA if samples were not collected during a cruise

xls_convert(data=all, project, cruise, version = "v1.0")